### Problem statement 
Companies face the issue of identifying communication networks when it comes to project or supply chain management.
Inherently this could lead to unintended dependencies or issues that may arise over time. 
Uncovering discussions over time by user allows businesses to act proactively to mitigate business risk.

Identify and Quantify network/connectivity significance. 
enabling the mitigation of business continuity risk through better resource allocation.

### Solution
Risk profiling would be done on two levels:
1. Face/superficial
2. Topical/knowledge

The profiles would indicate the following metrics:
    - Degree 
    - Topical discussion over time

Through the dossiers management would be able to better identify knowledge and cultural dependencies.

Applictaion to supply chain:
Feature: transaction volume/value, units. 
- identification and ranking of suppliers through communique (sentiment/reliability)
- identification of transaction volume through suppliers
- identification of dependency risk by production process.


In [1]:
import numpy as np 
import pandas as pd
from time import time
import re
import random
import pickle
from datetime import datetime, timedelta

#timing for whole notebook
tt = time()

In [2]:
df = pd.read_csv('D:/Capstone/dataset/emails.csv')

In [3]:
#username - for later use
df['user']=df['file'].str.extract(r"([a-z]+-[^\/]+)")
#extract folder name - for later use
df['folder']=df['file'].str.extract(r"/(.*)\.")
df['folder']=df['folder'].str.lower()
#drop file, no longer needed
df=df.drop(columns ='file')
#replace all \n with spaces
df = df.replace(r'(\n)',' ', regex=True) 
#extract emails
df['from'] = df['message'].str.extract(r'From: ([\S]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)')

In [4]:
t = time()
df['from2']=df[df['from'].isna()]['message'].str.extract(r'From: .*? \<([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)\> To:')
df["from"].fillna(df['from2'],inplace=True)
df['from3']=df[df['from'].isna()]['message'].str.extract(r'From: .*? \<([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)\> ')
df["from"].fillna(df['from3'],inplace=True)
df=df.drop(columns=['from2','from3'])
df.loc[df['from'].isna(),['from']]=df[df['from'].isna()]['message'].str.extract(r'From: .* \<([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)\> To:')
df['from'] = df['from'].replace(r'\<','',regex = True) 
df['from']=df['from'].str.lower()
#drop no.address@enron.com
df=df[df['from']!='no.address@enron.com']

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))
#checked all emails (up to emails with 106 characters) to verify validity of email extraction.

Time to clean up everything: 0.02 mins


In [5]:
#identify sent or received
df['type']=df['folder'].str.extract(r'(sent)')
df['type'].fillna('received',inplace=True)

In [6]:
#identify title/subject for possible topic attribution
df['subject'] = df['message'].str.extract(r'Subject:\s(.*)\sMime')
df['subject'] = df['subject'].replace(r'(Cc:.*|Mime-.*)\s','',regex = True) 

In [7]:
df['subject2']=df[df['subject'].isna()]['message'].str.extract(r'Subject:(.*) -Version')
df["subject"].fillna(df['subject2'],inplace=True)
df=df.drop(columns=['subject2'])

In [8]:
#Cleaning subjects
df['subject'] = df['subject'].replace(r'[Rr][Ee]:|[Ff][Ww]:|[Tt][Oo]:|(\d+)',' ',regex = True)
df['subject'] = df['subject'].replace(r'[\w\.-]+@[\w\.-]+','',regex = True)
df['subject'] = df['subject'].replace(r'[^A-Za-z0-9\.]+',' ',regex = True)

In [9]:
#fill blank subjects
df['subject'].fillna(value=str('no subject'), inplace=True)
df.loc[df['subject']==' ',['subject']]='no subject'
df.loc[df['subject']=='',['subject']]='no subject'

In [10]:
t = time()
#extract date & time
df['date'] = df['message'].str.extract(r'Date: (\w+, \d+ \w+ \d+ .*?)[+-]')
#extract bcc
df['bcc'] = df['message'].str.extract(r'X-bcc: (.*)X-Folder')
df['bcc'] = df['bcc'].replace(r'<(.*)>|(\s+){2}','',regex = True)
df.loc[df['bcc']==' ',['bcc']]='None'
df.bcc.fillna(value = 'None',inplace=True)

#remove CC, not important
df['message'] = df.replace(r'Cc: (.*) Mime',' ',regex = True)

df = df.replace(r'(\r)','',regex = True)
print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 0.36 mins


In [11]:
#checkpoint
print('number of rows:',df.shape[0])
print('number of bbc:',df[df.bcc!='None'].shape[0])
print('number of sent users:',df[df.type=='sent']['user'].value_counts().shape[0])

number of rows: 512289
number of bbc: 173
number of sent users: 149


In [12]:
user_list=df['user'].unique().tolist()
email_list=df['from'].unique().tolist()

In [13]:
def counta(df,cnt_what,by2_criteria,by2,by1,by_list):
    return_list=[]
    count_list=[]
    for i in by_list:
        a=df[(df[by1]==i) & (df[by2]==by2_criteria)][cnt_what].value_counts()
        return_list.append(a)
        count_list.append(len(a))
    df = pd.DataFrame({'input_list': by_list,
                       'output_list': return_list,
                       'count_list': count_list})
    return df

def modea(df,rep_what,by2_criteria,by2,by1,by_list):
    return_list=[]
    for i in by_list:
        mode = df[(df[by1]==i)&(df[by2]== by2_criteria)][rep_what].mode()
        try:
            modea=mode[0]
            return_list.append(modea)
        except:
            modea='NO EMAIL'
            return_list.append(modea)
    df = pd.DataFrame({'input_list': by_list,
                       'output_list': return_list
                       })
    return df    

In [14]:
check=counta(df,'from','sent','type','user',user_list)
check[check['count_list']>2]

,input_list,output_list,count_list
0,allen-p,phillip.allen@enron.com 1203 k..allen@enron...,4
7,beck-s,sally.beck@enron.com 2518 patti.thompso...,3
9,blair-l,lynn.blair@enron.com 1009 shelley.corm...,7
12,campbell-l,larry.campbell@enron.com 600 f..campbell@en...,3
14,cash-m,michelle.cash@enron.com 1100 twanda.sweet@e...,3
22,delainey-d,david.delainey@enron.com 1518 kay.chapman...,4
34,gay-r,randall.gay@enron.com 354 rob.gay@enron.com...,3
38,giron-d,darron.giron@enron.com 1581 c..giron@enro...,3
42,haedicke-m,mark.haedicke@enron.com 913 e..haedicke...,6
49,hodge-j,t..hodge@enron.com 61 john.hodge@enro...,6


In [15]:
user_key = modea(df,'from','sent','type','user',user_list)
user_key.head()

,input_list,output_list
0,allen-p,phillip.allen@enron.com
1,arnold-j,john.arnold@enron.com
2,arora-h,harry.arora@enron.com
3,badeer-r,robert.badeer@enron.com
4,bailey-s,susan.bailey@enron.com


In [16]:
user_key.columns=['user','email']
df= df.merge(user_key,on='user')
df[df['email']=='NO EMAIL']['user'].value_counts()

harris-s    514
Name: user, dtype: int64

In [17]:
#checkpoint
print('number of rows:',df.shape[0])
print('number of bbc:',df[df.bcc!='None'].shape[0])
print('number of sent users:',df[df.type=='sent']['user'].value_counts().shape[0])

number of rows: 512289
number of bbc: 173
number of sent users: 149


In [18]:
t = time()
#Attempt to parse message
df['body1'] = df['message'].str.extract(r'\.[pP][sS][tT](.*)')
df['body2'] = df['message'].str.extract(r'\.nsf(.*)')
df['body1'] = df['body1'].fillna('**NAN**')
df['body2'] = df['body2'].fillna('**NAN**')
df['body'] = df['body1'].map(str)+df['body2'].map(str)
df=df.drop(columns=['body1','body2'])
df['body3']=df[df['body']=='**NAN****NAN**']['message'].str.extract(r'X-FileName: (.*)')
df['body3'] = df['body3'].fillna('**NAN**')
df['body'] = df['body'].map(str)+df['body3'].map(str)
df=df.drop(columns='body3')
print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 0.1 mins


In [19]:
t = time()
df['body'] = df['body'].replace(r' (http|ftp|https)\:\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?',
                                '',regex = True)
df['body'] = df['body'].replace(r'([Ww]{3}.*?\s)','',regex = True)
df['body'] = df['body'].replace(r'(-+ Forwarded.*?Subject:)|(<.*?Subject:)|(From:.*?Content-Length:)|(From:.*?Subject:)',
                                '',regex = True)
print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 3.29 mins


In [20]:
df['body'] = df['body'].replace(r'(-+Original Message-+|\[IMAGE\])|(To:.*?Subject:)','',regex = True)
df['body'] = df['body'].replace(r'(---+|___+|~~~+|===+|\.\.\.+|\*\*\*+|\+\+\++|\?\?\?+)',' ',regex = True)

In [21]:
df['body'] = df['body'].replace(r'(=20|=01|=3D|=09)','',regex = True) 
df['body'] = df['body'].replace(r'(\*\*NAN\*\*)','',regex = True)
df['body'] = df['body'].replace(r'(\*\*NAN|NAN\*\*|NAN)','',regex = True)

In [22]:
#remove signatures
df['body'] = df['body'].replace(r'This e[\s\-]mail is the property of Enron Corp. and or its relevant affiliate and may contain confidential and privileged material for the sole use of the intended recipient s . Any review use distribution or disclosure by others is strictly prohibited. If you are not the intended recipient or authorized to receive for the recipient please contact the sender or reply to Enron Corp. at and delete all copies of the message. This e mail and any attachments hereto are not intended to be an offer or an acceptance and do not create or evidence a binding and enforceable contract between Enron Corp. or any of its affiliates and the intended recipient or any other party and may not be relied on by anyone as the basis of a contract by estoppel or otherwise. Thank you.','',regex = True)
df['body'] = df['body'].replace(r'This email and any files transmitted with it from the ElPaso Corporation are confidential and intended solely for the use of the individual or entity to whom they are addressed. If you have received this email in error please notify the sender.','',regex = True) 
df['body'] = df['body'].replace(r'Sent from my BlackBerry Wireless Handheld www.BlackBerry.net','',regex = True)
#join 2 caps together
df['body'] = df['body'].replace(r'[A-Z](/s)[A-Z]','',regex = True)

In [23]:
t = time()
#remove document references
df['body'] = df['body'].replace(r'([A-za-z0-9]+?.([Dd][Oo][Cc]|[Pp][Dd][Ff]|[Tt][Xx][Tt]|[Jj][Pp][Gg]|[Xx][Ll][Ss]))','',regex = True)
#remove emails in line
df['body'] = df['body'].replace(r'[\w\.-]+@[\w\.-]+','',regex = True)
#remove time, various contact numbers.(added |\d+ to remove digits for NLP processing, removed for data retrival purposes.)
df['body'] = df['body'].replace(r'(\d+:\d+ [Aa][Mm])|(\d+:\d+ [Pp][Mm])|(\d+:\d+ [PpAa]\.[Mm]\.)|\\t|\d+|Tel:|Fax:|Email:|[Rr][Ee]:|[Ff][Ww]:|[Tt][Oo]:',
                                '',regex = True)
df['body'] = df['body'].replace(r'[^A-Za-z0-9\.\']+',' ',regex = True)
print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 5.96 mins


In [27]:
#checkpoint
print('number of rows:',df.shape[0])
print('number of bbc:',df[df.bcc!='None'].shape[0])
print('number of sent users:',df[df.type=='sent']['user'].value_counts().shape[0])

number of rows: 473749
number of bbc: 171
number of sent users: 149


In [25]:
#emails longer than 1k words are mostly mailers and spam
df['word_count']=df['body'].apply(lambda x: len(x.split(' ')))
display(df[df['word_count']>1000].shape[0])

#keep emails 5 - 999 words
df=df[df['word_count']<1000]
df=df[df['word_count']>4]

16535

In [26]:
#identify non-frequent senders - meaningfulness of contact.
email_counts=df['from'].value_counts().rename('email_counts')

df=df.merge(email_counts.to_frame(),
             left_on='from',
             right_index=True)

#drop people who send less than 3 messages
df=df[df['email_counts']>2]
#df=df.drop(columns='email_counts')

#Drop unnecessary columns
df.reset_index(drop=True, inplace=True)
df['body'] = df['body'].replace(r'\.','',regex = True) 

In [28]:
df.to_csv('D:/Capstone/dataset/emails_total_preprocessed_raw.csv')

In [ ]:
##NLP is done from documents 2-5.
##Topic Attribution 6 & 7
##Stats Extraction 8 (group level analytics)
##Building N4J database 9
##Identify Position of employees 10
##Test relationships between degree, topics and position 11, 12
##Individual Dashboarding

In [ ]:
#Drop duplicate emails
df.drop_duplicates(subset ="body", keep='first', inplace = True)
print(df.shape)

In [ ]:
#Checkpoint- Check bcc rows, check outbox.
print(df.shape)
print(df[df.bcc!='None'].shape)
print(df[df.type=='sent']['user'].value_counts().shape)

In [ ]:
df.info()

In [ ]:
df.to_csv('D:/Capstone/dataset/emails_v1.csv')

In [ ]:
sample = df.sample(n=5000)
sample.to_csv('D:/Capstone/dataset/Enronsamplev1.csv')

In [ ]:
class StrToBytes:
    def __init__(self, fileobj):
        self.fileobj = fileobj
    def read(self, size):
        return self.fileobj.read(size).encode()
    def readline(self, size=-1):
        return self.fileobj.readline(size).encode()

In [ ]:
### Load the dictionary containing the dataset
with open('D:/Capstone/dataset/POI/final_project_dataset.pkl', 'r') as data_file:
    data_dict = pickle.load(StrToBytes(data_file))

# dict to dataframe
df = pd.DataFrame.from_dict(data_dict, orient='index')
df.replace('NaN', np.nan, inplace = True)

In [ ]:
df.to_csv('D:/Capstone/dataset/EnronEmployee.csv')
print('Time to clean up everything: {} mins'.format(round((time() - tt) / 60, 2)))